# Alcohol & Substance Abuse Related Mortality Predictions

Datasets for cleaning will include:
1) IMHE County Level Mortality Rates, 1980 - 2014 (four mortality causes: alcohol use disorders, substance use disorders, self-harm, and interpersonal violence
2) IMHE County Level Alcohol Use and Prevalence Data, 2002 - 2012 (includes prevalence of alcohol use by: any, heavy & binge, as well as proportion of drinkers who are heavy drinkers and binge drinkers)
3) County Level Unemployment Data: 

Utilizing IMHE data, build a model to predict alcohol and substance abuse related mortality at the county level. 

Combine with demographics (Census/ACS) to create predictive models / explore explanatory variables. 

[CDC Wonder API Tutorial](https://www.youtube.com/watch?v=ttchrtCZ46Y)

In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os

In [2]:
alcohol_death_files = os.listdir('../data/alcohol_substance_deaths/')
alcohol_prev_files = os.listdir('../data/alcohol_use_prev/')

In [3]:
df = pd.read_csv('../data/alcohol_substance_deaths/' + alcohol_death_files[0])
df.head(3)

,measure_id,measure_name,location_id,location_name,FIPS,cause_id,cause_name,sex_id,sex,age_id,age_name,year_id,metric,mx,lower,upper
0,1,Deaths,536,Illinois,17,560,Alcohol use disorders,1,Male,27,Age-standardized,1980,Rate,4.133318,3.728972,4.699632
1,1,Deaths,536,Illinois,17,560,Alcohol use disorders,1,Male,27,Age-standardized,1981,Rate,4.255357,3.876988,4.759205
2,1,Deaths,536,Illinois,17,560,Alcohol use disorders,1,Male,27,Age-standardized,1982,Rate,3.900788,3.563317,4.383266


## 1. Read in the IMHE Mortality Data, create dataframe
The IMHE mortality dataset came from ***LINK HERE***, and shows the mortality rate by county, year and sex (Male, Female, Both) for self-harm, alcohol use disorders, drug use disorders and interpersonal violence. 

I want to first create a function to process the whole thing. Later when I decide which to focus on, I'll create a smaller data frame that is just one of the mortality categories. 

In [4]:
# function to process IMHE alcohol data
def process_mortality_data(file):
    state = file.split('_')[-2].title()
    full_path = '../data/alcohol_substance_deaths/' + file


    
    #read in csv
    df = pd.read_csv(full_path, dtype={'FIPS': object})
    # drop rows that are sums of whole state
    df = df[(df['location_name'] != state) & (df['FIPS'].isna() == False)]
    
    #recase cols to save memory and so we can create a UID later
    # UID is formatted: 'FIPS-cause_id-sex_id-year_id'
    #df[['cause_id', 'sex', 'cause_name', 'location_name', 'FIPS']] = df[['location_name', 'FIPS', 'cause_id', 'sex', 'cause_name']].astype('category')
    #df['UID'] = df['FIPS'] + '-' + df['cause_id'] + '-' + df['sex_id'] + '-' + df['year_id']
    df['state'] = state
    
    # fix the state names
    # North Carolina
    df.replace('Northcarolina', 'North Carolina', inplace=True)
    df.replace('Northdakota', 'North Dakota', inplace=True)
    df.replace('Southcarolina', 'South Carolina', inplace=True)
    df.replace('Southdakota', 'South Dakota', inplace=True)
    df.replace('Newyork', 'New York', inplace=True)
    df.replace('Newjersey', 'New Jersey', inplace=True)
    df.replace('Newhampshire', 'New Hampshire', inplace=True)
    df.replace('Newmexico', 'New Mexico', inplace=True)
    df.replace('Westvirginia', 'West Virginia', inplace=True)
    df.replace('Rhodeisland', 'Rhode Island', inplace=True)
    df.replace('Dc', 'D.C.', inplace=True)
    
    df.drop(columns=['measure_name', 'cause_id', 'metric', 'sex_id', 'location_id', 'age_id', 'age_name', 'lower', 'upper'], inplace=True)
    
    #create UID columns for later joining with other dataframes
    df['UID'] = df['FIPS'] + '-' + df['year_id'].astype('string') 
    df['UID_2'] = df['FIPS'] + '-' + df['sex'] + '-' + df['year_id'].astype('string')
    
    
    return df

In [5]:
# process mortality data
full_mortality_df = [process_mortality_data(file) for file in alcohol_death_files]
full_mortality_df = pd.concat(full_mortality_df)
full_mortality_df.drop(columns=['measure_id', 'measure_ID'], inplace=True)

full_mortality_df['FIPS'] = full_mortality_df['FIPS'].str.zfill(5)

In [6]:
full_mortality_df.memory_usage(deep=True) / 1_000_000

Index            10.59408
location_name    94.07076
FIPS             82.10412
cause_name       98.65737
sex              81.66270
year_id          10.59408
mx               10.59408
state            86.19114
UID              88.57884
UID_2            96.08298
dtype: float64

In [7]:
# creating a Unique Identifier to join later with other climate dfs
subInj_df_compression_opts = dict(method='zip',
                        archive_name='alcohol_sub_inj_mortality_1980_2014.csv')

full_mortality_df.to_csv('../data/cleaned/alcohol_sub_inj_mortality_1980_2014.zip', index=False, compression=subInj_df_compression_opts)

In [8]:
# export full mortality df to CSV
# full_mortality_df.to_csv('data/cleaned/alcohol_sub_inj_mortality_1980_2014.csv', index=False) --> This was 146 MB, so I compressed for upload to github

#grab just self-harm
selfharm_df = full_mortality_df[full_mortality_df['cause_name'] == 'Self-harm']

#export just alcohol to csv as well
selfharm_df.to_csv('../data/cleaned/selfharm_mortality_1980_2014.csv', index=False)

#### We will also create a quick dictionary of FIPS codes that will allow us to append this data to other dataframes that might have county name and state but not the FIPS code. We need FIPS for maps & streamlit later

In [9]:
selfharm_df['county_state'] = selfharm_df['location_name'] + ", " + selfharm_df['state']
selfharm_unique = selfharm_df.drop_duplicates(subset=['county_state'])

# create dictionary of county name / FIPS values
#https://stackoverflow.com/questions/18012505/python-pandas-dataframe-columns-convert-to-dict-key-and-value
county_fips = dict(zip(selfharm_unique['county_state'], selfharm_unique['FIPS']))

<ipython-input-9-c868c4f2ca44>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selfharm_df['county_state'] = selfharm_df['location_name'] + ", " + selfharm_df['state']


## 2. Read in Alcohol Prevalence Data

Study Link: http://www.healthdata.org/research-article/drinking-patterns-us-counties-2002-2012 

An excel file from IMHE, tab names for reading in include:
- **Any** : "Any" drinking is defined as at least one drink of any alcoholic bevarage in the past 30 days
- **Heavy** : "Heavy" drinking is defined as the consumption, on average, of more than one drink per day for women or two drinks per day for men in the past 30 days
- **Binge** : "Binge" drinking is defined as the consumption of more than four drinks for women or five drinks for men on a single occasion at least once in the past 30 days
- **Prop. Heavy** : Proportion of All Drinkers Who Are Heavy Drinkers by year and sex, 2005-2012
- **Prop. Binge** : Proportion of All Drinkers Who Are Binge Drinkers by year and sex

Create one file that has years, counties and sex, and each column represents `any`, `heavy`, `prop_heavy`, `binge`, and `prop_binge`

In [10]:
def clean_alcohol_use(file, tab_name):
    df = pd.read_excel(file, tab_name)
    
    #get rid of period/space in tab_name
    #tab_name = tab_name.replace('.', '')
    #print(tab_name)
    
    # all these files have state total and national total -- we just want county totals
    # drop rows where "County" string is not in Location column
    df = df[df['Location'].str.contains('County')]
    df.reset_index(inplace=True)
    
    # drop unnecessary columns heavy and prop heavy are only 2005 to 2012, so we're dropping different columns
    if tab_name == 'Heavy':
        df.drop(columns=['index', 'Percent Change 2005-2012, Both Sexes', 'Percent Change 2005-2012, Females', 'Percent Change 2005-2012, Males'], inplace=True)
    elif tab_name == 'Prop. Heavy':
        df.drop(columns=['index', 'Percent Change 2005-2012, Both Sexes', 'Percent Change 2005-2012, Females', 'Percent Change 2005-2012, Males'], inplace=True)
    elif tab_name == 'Prop. Binge':
        df.drop(columns=['index', 'Percent Change 2002-2012, Both Sexes', 'Percent Change 2002-2012, Females', 'Percent Change 2002-2012, Males', 'Percent Change 2005-2012, Both Sexes', 'Percent Change 2005-2012, Females', 'Percent Change 2005-2012, Males'], inplace=True)
    else:
        df.drop(columns=['index', 'Percent Change 2002-2012, Both Sexes', 'Percent Change 2002-2012, Females', 'Percent Change 2002-2012, Males', 'Percent Change 2005-2012, Both Sexes', 'Percent Change 2005-2012, Females', 'Percent Change 2005-2012, Males'], inplace=True)
    
    # unmelt the columns
    # binge drinking and self-harm
    tab_name = tab_name.replace('. ', '_')
    melted_val_col_name = "alcohol_" + tab_name.lower() 
    df = df.melt(id_vars=['State', 'Location'], var_name='year_metric_category', value_name=melted_val_col_name)

    # after "melting", create new columns for year & sex
    df['year'] = df['year_metric_category'].str[0:4]
    df['sex'] = df['year_metric_category'].str[5:] 
    df.replace({'Both Sexes': 'Both', 'Males': 'Male', 'Females': 'Female'}, inplace=True)

    # create new col with county_state so we can join with another df to get the FIPS : this will be used in the streamlit app
    df['county_state'] = df['Location'] + ", " + df['State']
    
    #now add FIPS to the binge drinking data
    df['FIPS'] = df['county_state'].map(county_fips)
    
    #create UID for joining these to other dfs
    df['UID'] = df['FIPS'] + '-' + df['year'].astype('string') 
    df['UID_2'] = df['FIPS'] + '-' + df['sex'] + '-' + df['year'].astype('string')
    
    #rename some columns
    df.rename(columns={'State': 'state', 'Location': 'county_name'}, inplace=True)
    
    
    #reorder columns
    df = df[['FIPS', 'county_name', 'state', 'county_state', 'year', 'sex', 'year_metric_category', 'UID', 'UID_2',melted_val_col_name]]
    
    return df
    


In [11]:
alcohol_use_file = '../data/alcohol_use_prev/IHME_USA_COUNTY_ALCOHOL_USE_PREVALENCE_2002_2012_NATIONAL_Y2015M04D23.XLSX'

#read in all files using cleaned function
df_any = clean_alcohol_use(alcohol_use_file, 'Any')
df_heavy = clean_alcohol_use(alcohol_use_file, 'Heavy')
df_prop_heavy = clean_alcohol_use(alcohol_use_file, 'Prop. Heavy')
df_binge = clean_alcohol_use(alcohol_use_file, 'Binge')
df_prop_binge = clean_alcohol_use(alcohol_use_file, 'Prop. Binge')

In [12]:
# now we need to join files. The goal is one file that has a new column for every alcohol use category
alcohol_heavy_prop_heavy = pd.merge(df_heavy, df_prop_heavy, on='UID_2', how='left', suffixes=('', '_y'))
#drop cols
alcohol_heavy_prop_heavy.drop(columns=['FIPS_y', 'county_name_y', 'state_y', 'county_state_y', 'year_y', 'sex_y', 'year_metric_category_y', 'UID_y'], inplace=True)

In [13]:
#do the same to join binge/prop binge
alcohol_binge_prop_binge = pd.merge(df_binge, df_prop_binge, on='UID_2', how='left', suffixes=('', '_y'))
alcohol_binge_prop_binge.drop(columns=['FIPS_y', 'county_name_y', 'state_y', 'county_state_y', 'year_y', 'sex_y', 'year_metric_category_y', 'UID_y'], inplace=True)

In [14]:
alcohol_heavy_prop_heavy.memory_usage(deep=True) / 1_000_000

Index                    3.451584
FIPS                    15.949776
county_name             33.791280
state                   28.049880
county_state            38.111520
year                    26.318328
sex                     26.605960
year_metric_category    29.913728
UID                     19.187016
UID_2                   19.591888
alcohol_heavy            3.451584
alcohol_prop_heavy       3.451584
dtype: float64

In [14]:
# the merged files are pretty big, so the big ones should be zipped
alcohol_heavy_compression_opts = dict(method='zip',
                        archive_name='alcohol_use_heavy_prop_heavy_2005_2012.csv')
alcohol_binge_compression_opts = dict(method='zip',
                        archive_name='alcohol_use_binge_prop_binge_2005_2012.csv')

#write to csv
alcohol_heavy_prop_heavy.to_csv('../data/cleaned/alcohol_use_heavy_prop_heavy_2005_2012.zip', index=False, compression=alcohol_heavy_compression_opts)
alcohol_binge_prop_binge.to_csv('../data/cleaned/alcohol_use_binge_prop_binge_2005_2012.zip', index=False, compression=alcohol_binge_compression_opts)
df_any.to_csv('../data/cleaned/alcohol_use_any_2002_2012.csv', index=False)

## 3. Unemployment Data

These had some WEIRD formatting when I downloaded them. You can see below. I opened up the file in excel, and it was just someone trying to be fancy with titles/formatting. It was ultimately quicker to correct the header issues manually in Excel and save the new csvs in a separate folder. I still had some processing to do after that my manual correction though. 

In [15]:
unemployment = os.listdir('../data/unemployment_excel/')

In [16]:
test_file = '../data/unemployment_excel/' + unemployment[0]
unemployment_test_df = pd.read_excel(test_file, header=1)

In [17]:
unemployment_test_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,State,County,NaN,NaN,NaN,NaN,NaN,NaN,Unemploy-
1,LAUS,FIPS,FIPS,NaN,NaN,NaN,Labor,NaN,NaN,ment Rate
2,Code,Code,Code,County Name/State Abbreviation,Year,NaN,Force,Employed,Unemployed,(%)
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CN0100100000000,01,001,"Autauga County, AL",1995,NaN,19632,18525,1107,5.6


In [19]:
unemployment_files = os.listdir('../data/unemployment_csv/')
test_file = '../data/unemployment_csv/' + unemployment_files[0]
test_df = pd.read_csv(test_file)
test_df.head(2)

,LAUS_CODE,St_FIPS,County_FIPS,county_state_name,Year,labor_force,employed,unemployed,unemployment_rate
0,CN0100100000000,1.0,1.0,"Autauga County, AL",1999.0,"22,533","21,678",855,3.8
1,CN0100300000000,1.0,3.0,"Baldwin County, AL",1999.0,"71,240","68,880","2,360",3.3


The columns for state and county FIPS area all messed up, and I need those to match up to the FIPS codes in other datasets...luckily, the FIPS code is also contained in the LAUS_CODE String. The first five digits after the CN are the FIPS code, so that's perf. I'll likely replace state FIPS with just the State name, and create a new column that also has just the county name (instead of county, state) for use later in the Streamlit app. 

Also there's lots of empty rows, and we don't need data where state is 'PR', or nan. Where ST is 'ia', we replace with DC, since this is the District of Columbia.

I'm going to write a function to clean up this data.

In [20]:
# function to clean the unemployment data
def clean_unemployment(file):
    full_path = '../data/unemployment_csv/' + file

    
    #read in csv
    df = pd.read_csv(full_path, dtype={'Year': 'Int64', 'unemployment_rate': 'float64' } )
    
    # drop Null rows
    df.dropna(inplace=True)
    # drop fips cols, we will get FIPS from the LAUS code
    df.drop(columns=['St_FIPS', 'County_FIPS'], inplace=True)
    
    #take care of the weird formatting in labor force, employed, unemployed columns
    df['labor_force'] = df['labor_force'].str.strip()
    df['employed'] = df['employed'].str.strip()
    df['unemployed'] = df['unemployed'].str.strip()
    
    df['labor_force'] = df['labor_force'].str.replace(',', '')
    df['employed'] = df['employed'].str.replace(',', '')
    df['unemployed'] = df['unemployed'].str.replace(',', '')
    
    # drop the N.A.
    df = df[(df['labor_force'] != 'N.A.') & (df['employed'] != 'N.A.') & (df['unemployed'] != 'N.A.')]
    
    # cast labor_force, employed, unemployed as ints
    df['labor_force'] = df['labor_force'].astype(int)
    df['employed'] = df['employed'].astype(int)
    df['unemployed'] = df['unemployed'].astype(int)
    
    #create new FIPS column
    df['FIPS'] = df['LAUS_CODE'].str[2:7]
    df['state'] = df['county_state_name'].str[-2:]
    
    df['state'].replace('ia', 'DC', inplace=True)
    
    # get rid of PR
    df = df[df['state'] != 'PR']
    df.rename(columns={'Year': 'year'}, inplace=True)
    
    # first create a UID that is 'ST-FIPS-YEAR' -- ex. 'AL-01001-1999'
    df['UID'] = df['FIPS'] + '-' + df['year'].astype('string')
    
    #reorder the columns to make it nice
    df = df[['FIPS', 'UID', 'county_state_name', 'state', 'year', 'labor_force', 'employed', 'unemployed', 'unemployment_rate']]
    
    
    return df

In [21]:
# test our function to see if it works
clean_unemployment(unemployment_files[14]) # it works!!

,FIPS,UID,county_state_name,state,year,labor_force,employed,unemployed,unemployment_rate
0,01001,01001-2006,"Autauga County, AL",AL,2006,24425,23619,806,3.3
1,01003,01003-2006,"Baldwin County, AL",AL,2006,79806,77263,2543,3.2
2,01005,01005-2006,"Barbour County, AL",AL,2006,10713,10110,603,5.6
3,01007,01007-2006,"Bibb County, AL",AL,2006,8858,8489,369,4.2
4,01009,01009-2006,"Blount County, AL",AL,2006,26799,25939,860,3.2
...,...,...,...,...,...,...,...,...,...
3134,56037,56037-2006,"Sweetwater County, WY",WY,2006,23606,23051,555,2.4
3135,56039,56039-2006,"Teton County, WY",WY,2006,14113,13774,339,2.4
3136,56041,56041-2006,"Uinta County, WY",WY,2006,11092,10776,316,2.8
3137,56043,56043-2006,"Washakie County, WY",WY,2006,4208,4064,144,3.4


In [22]:
full_unemployment_df = [clean_unemployment(file) for file in unemployment_files]
full_unemployment_df = pd.concat(full_unemployment_df)

In [23]:
full_unemployment_df.head(2)

,FIPS,UID,county_state_name,state,year,labor_force,employed,unemployed,unemployment_rate
0,01001,01001-1999,"Autauga County, AL",AL,1999,22533,21678,855,3.8
1,01003,01003-1999,"Baldwin County, AL",AL,1999,71240,68880,2360,3.3


In [24]:
# write it out to a csv in the cleaned data folder
full_unemployment_df.to_csv('../data/cleaned/unemployment_1990_2019.csv', index=False)

# Combining Alcohol Use Data with Self-Harm Mortality

Data Science and Data Analysis are an interative proces. While the first portion of this notebook involved cleaning disparate datasets, now that I have decided on a problem statement, I will combine the necessary datasets to make for later visualizing and modeling.  

I will be combining the self-harm mortality by county, and the alcohol use dataframes.


In [25]:
# self-harm and any alcohol use
harm_alcohol_any = pd.merge(left=selfharm_df, right=df_any, on='UID_2', how='left', suffixes=('', '_y'))
harm_alcohol_any.drop(columns=['FIPS_y', 'county_name', 'state_y', 'county_state', 'year', 'sex_y', 'year_metric_category', 'UID_y'], inplace=True)

# now we need to join files. The goal is one file that has a new column for every alcohol use category
#drop cols
harm_alcohol_any

,location_name,FIPS,cause_name,sex,year_id,mx,state,UID,UID_2,county_state_y,alcohol_any
0,Adams County,17001,Self-harm,Male,1980,22.993767,Illinois,17001-1980,17001-Male-1980,NaN,NaN
1,Adams County,17001,Self-harm,Male,1981,22.059783,Illinois,17001-1981,17001-Male-1981,NaN,NaN
2,Adams County,17001,Self-harm,Male,1982,23.981794,Illinois,17001-1982,17001-Male-1982,NaN,NaN
3,Adams County,17001,Self-harm,Male,1983,22.824779,Illinois,17001-1983,17001-Male-1983,NaN,NaN
4,Adams County,17001,Self-harm,Male,1984,22.555860,Illinois,17001-1984,17001-Male-1984,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
331060,Wyoming County,54109,Self-harm,Both,2010,21.706674,West Virginia,54109-2010,54109-Both-2010,"Wyoming County, West Virginia",23.1
331061,Wyoming County,54109,Self-harm,Both,2011,22.570620,West Virginia,54109-2011,54109-Both-2011,"Wyoming County, West Virginia",23.3
331062,Wyoming County,54109,Self-harm,Both,2012,22.670492,West Virginia,54109-2012,54109-Both-2012,"Wyoming County, West Virginia",23.5
331063,Wyoming County,54109,Self-harm,Both,2013,23.199732,West Virginia,54109-2013,54109-Both-2013,NaN,NaN


In [26]:
# join heavy_prop_heavy with self harm
harm_alcohol_heavy_prop_heavy = pd.merge(left=selfharm_df, right=alcohol_heavy_prop_heavy, on='UID_2', how='left', suffixes=('', '_y'))
harm_alcohol_heavy_prop_heavy.drop(columns=['FIPS_y', 'county_name', 'state_y', 'county_state', 'year', 'sex_y', 'year_metric_category', 'UID_y'], inplace=True)
harm_alcohol_heavy_prop_heavy

,location_name,FIPS,cause_name,sex,year_id,mx,state,UID,UID_2,county_state_y,alcohol_heavy,alcohol_prop_heavy
0,Adams County,17001,Self-harm,Male,1980,22.993767,Illinois,17001-1980,17001-Male-1980,NaN,NaN,NaN
1,Adams County,17001,Self-harm,Male,1981,22.059783,Illinois,17001-1981,17001-Male-1981,NaN,NaN,NaN
2,Adams County,17001,Self-harm,Male,1982,23.981794,Illinois,17001-1982,17001-Male-1982,NaN,NaN,NaN
3,Adams County,17001,Self-harm,Male,1983,22.824779,Illinois,17001-1983,17001-Male-1983,NaN,NaN,NaN
4,Adams County,17001,Self-harm,Male,1984,22.555860,Illinois,17001-1984,17001-Male-1984,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
331060,Wyoming County,54109,Self-harm,Both,2010,21.706674,West Virginia,54109-2010,54109-Both-2010,"Wyoming County, West Virginia",4.3,18.4
331061,Wyoming County,54109,Self-harm,Both,2011,22.570620,West Virginia,54109-2011,54109-Both-2011,"Wyoming County, West Virginia",4.6,19.8
331062,Wyoming County,54109,Self-harm,Both,2012,22.670492,West Virginia,54109-2012,54109-Both-2012,"Wyoming County, West Virginia",4.4,18.8
331063,Wyoming County,54109,Self-harm,Both,2013,23.199732,West Virginia,54109-2013,54109-Both-2013,NaN,NaN,NaN


In [27]:
# now join self harm and binge_prop_binge
harm_alcohol_binge_prop_binge = pd.merge(left=selfharm_df, right=alcohol_binge_prop_binge, on='UID_2', how='left', suffixes=('', '_y'))
harm_alcohol_binge_prop_binge.drop(columns=['FIPS_y', 'county_name', 'state_y', 'county_state', 'year', 'sex_y', 'year_metric_category', 'UID_y'], inplace=True)
harm_alcohol_binge_prop_binge

,location_name,FIPS,cause_name,sex,year_id,mx,state,UID,UID_2,county_state_y,alcohol_binge,alcohol_prop_binge
0,Adams County,17001,Self-harm,Male,1980,22.993767,Illinois,17001-1980,17001-Male-1980,NaN,NaN,NaN
1,Adams County,17001,Self-harm,Male,1981,22.059783,Illinois,17001-1981,17001-Male-1981,NaN,NaN,NaN
2,Adams County,17001,Self-harm,Male,1982,23.981794,Illinois,17001-1982,17001-Male-1982,NaN,NaN,NaN
3,Adams County,17001,Self-harm,Male,1983,22.824779,Illinois,17001-1983,17001-Male-1983,NaN,NaN,NaN
4,Adams County,17001,Self-harm,Male,1984,22.555860,Illinois,17001-1984,17001-Male-1984,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
331060,Wyoming County,54109,Self-harm,Both,2010,21.706674,West Virginia,54109-2010,54109-Both-2010,"Wyoming County, West Virginia",9.8,42.4
331061,Wyoming County,54109,Self-harm,Both,2011,22.570620,West Virginia,54109-2011,54109-Both-2011,"Wyoming County, West Virginia",10.2,43.8
331062,Wyoming County,54109,Self-harm,Both,2012,22.670492,West Virginia,54109-2012,54109-Both-2012,"Wyoming County, West Virginia",10.2,43.3
331063,Wyoming County,54109,Self-harm,Both,2013,23.199732,West Virginia,54109-2013,54109-Both-2013,NaN,NaN,NaN


In [28]:
# write the cleaned/joined files out to csv
harm_alcohol_any.to_csv('../data/cleaned/selfharm_alcohol_joins/selfharm_join_alcohol_any.csv', index=False)
harm_alcohol_heavy_prop_heavy.to_csv('../data/cleaned/selfharm_alcohol_joins/selfharm_join_heavy_prop_heavy.csv', index=False)
harm_alcohol_binge_prop_binge.to_csv('../data/cleaned/selfharm_alcohol_joins/selfharm_join_binge_prop_binge.csv', index=False)

# Join alcohol use, self harm and unemployment
This will be a different dataframe, since we don't have unemployment by sex

In [29]:
#drop the male/female rows, in favor of only both, since we don't have unemployment by gender
harm_alcohol_any_both_sexes = harm_alcohol_any[harm_alcohol_any['sex'] == 'Both']
harm_alcohol_heavy_prop_heavy_both_sexes = harm_alcohol_heavy_prop_heavy[harm_alcohol_heavy_prop_heavy['sex'] == 'Both']
harm_alcohol_binge_prop_binge_both_sexes = harm_alcohol_binge_prop_binge[harm_alcohol_binge_prop_binge['sex'] == 'Both']

In [30]:
# join self harm/alcohol combo dfs with unemployment
harm_alcohol_any_with_unemployment = pd.merge(harm_alcohol_any_both_sexes, full_unemployment_df, on='UID', how='inner', suffixes=('', '_y'))
harm_alcohol_heavy_with_unemployment = pd.merge(harm_alcohol_heavy_prop_heavy_both_sexes, full_unemployment_df, on='UID', how='inner', suffixes=('', '_y'))
harm_alcohol_binge_with_unemployment = pd.merge(harm_alcohol_binge_prop_binge_both_sexes, full_unemployment_df, on='UID', how='inner', suffixes=('', '_y'))

In [31]:
harm_alcohol_any_with_unemployment

,location_name,FIPS,cause_name,sex,year_id,mx,state,UID,UID_2,county_state_y,alcohol_any,FIPS_y,county_state_name,state_y,year,labor_force,employed,unemployed,unemployment_rate
0,Adams County,17001,Self-harm,Both,1990,13.147436,Illinois,17001-1990,17001-Both-1990,NaN,NaN,17001,"Adams County, IL",IL,1990,34002,32335,1667,4.9
1,Adams County,17001,Self-harm,Both,1991,12.796851,Illinois,17001-1991,17001-Both-1991,NaN,NaN,17001,"Adams County, IL",IL,1991,33703,31819,1884,5.6
2,Adams County,17001,Self-harm,Both,1992,12.024267,Illinois,17001-1992,17001-Both-1992,NaN,NaN,17001,"Adams County, IL",IL,1992,34647,32304,2343,6.8
3,Adams County,17001,Self-harm,Both,1993,12.998694,Illinois,17001-1993,17001-Both-1993,NaN,NaN,17001,"Adams County, IL",IL,1993,35700,33575,2125,6.0
4,Adams County,17001,Self-harm,Both,1994,12.131929,Illinois,17001-1994,17001-Both-1994,NaN,NaN,17001,"Adams County, IL",IL,1994,35832,34506,1326,3.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70606,Wyoming County,54109,Self-harm,Both,2010,21.706674,West Virginia,54109-2010,54109-Both-2010,"Wyoming County, West Virginia",23.1,54109,"Wyoming County, WV",WV,2010,8406,7530,876,10.4
70607,Wyoming County,54109,Self-harm,Both,2011,22.570620,West Virginia,54109-2011,54109-Both-2011,"Wyoming County, West Virginia",23.3,54109,"Wyoming County, WV",WV,2011,8308,7595,713,8.6
70608,Wyoming County,54109,Self-harm,Both,2012,22.670492,West Virginia,54109-2012,54109-Both-2012,"Wyoming County, West Virginia",23.5,54109,"Wyoming County, WV",WV,2012,8377,7633,744,8.9
70609,Wyoming County,54109,Self-harm,Both,2013,23.199732,West Virginia,54109-2013,54109-Both-2013,NaN,NaN,54109,"Wyoming County, WV",WV,2013,8296,7537,759,9.1


In [32]:
# drop unwanted cols
harm_alcohol_any_with_unemployment.drop(columns=['FIPS_y', 'county_state_y', 'county_state_name', 'year'], inplace=True)
harm_alcohol_heavy_with_unemployment.drop(columns=['FIPS_y', 'county_state_y', 'county_state_name', 'year'], inplace=True)
harm_alcohol_binge_with_unemployment.drop(columns=['FIPS_y', 'county_state_y', 'county_state_name', 'year'], inplace=True)

In [33]:
# write to csv
harm_alcohol_any_with_unemployment.to_csv('../data/cleaned/selfharm_alcohol_unemployment_joins/harm_alcohol_any_with_unemployment.csv', index=False)
harm_alcohol_heavy_with_unemployment.to_csv('../data/cleaned/selfharm_alcohol_unemployment_joins/harm_alcohol_heavy_with_unemployment.csv', index=False)
harm_alcohol_binge_with_unemployment.to_csv('../data/cleaned/selfharm_alcohol_unemployment_joins/harm_alcohol_binge_with_unemployment.csv', index=False)

#### Now that the Data are cleaned, we can move on to modeling!